<a href="https://colab.research.google.com/github/Deadshot10/HousingP_LinearR/blob/main/House_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.autograd import Variable #this is used to create tensors that support gradient calculations
#the autograd class is used to calculate the jacobian vector product


In [ ]:
class LinearRegressionModel(torch.nn.Module):
  def _init_(self): # basically the init method works  like a constructor from c++ which assigns the attributes of a class to the object that is created. 
  #Here we create a class of our linear regression and the init method is called anytime an object is defined wrt this class.
  #to the init method,when we pass self,attributeval,attrval.. what we do is assign those vals to the predefined attributes that the object inherited from the class.
    super(LinearRegressionModel,self)._init_():
     self.Linear= torch.nn.Linear(1,1)#one in one out as we are using a data set with one variable to predict the other variable the dependent variable

  def forward(self,x):
    y_pred=self.linear(x)
    return y_pred

In [ ]:
our_model = LinearRegressionModel() #this is basically creating an our model object which belongs to the linear regression class
criterion = torch.nn.MSELoss(size_average=False) #this is used to define the loss function
optimizer = torch.optim.SGD(our_model.parameters(),lr=0.01) #this is used to decide the optimization method for our loss function. The learning rate is basically the rate at which
# the parameters of the loss function are changed as we progress towards approaching the local minima. In linear regression problems there is only single global minima

In [ ]:
for epoch in range(700):
  #first we will perform the forward pass
  #pass x to the model

  pred_y = our_model(x_data)
  loss = criterion(pred_y,y_data)

  # Zero gradients, perform a backward pass,
  # and update the weights.
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  print('epoch {}, loss {}'.format(epoch, loss.item()))